In [1]:
config = "config.maze2d"
dataset = "maze2d-large-v1"

In [3]:
class Args:
    def __init__(self, loader, savepath, dataset, horizon, normalizer, preprocess_fns, use_padding, max_path_length, renderer):
        self.loader = loader
        self.savepath = savepath
        self.dataset = dataset
        self.horizon = horizon
        self.normalizer = normalizer
        self.preprocess_fns = preprocess_fns
        self.use_padding = use_padding
        self.max_path_length = max_path_length
        self.renderer = renderer

args = Args(
    loader='datasets.sequence.GoalDataset',
    savepath='saved/',
    dataset='maze2d-large-v1',
    horizon=256,
    normalizer='LimitsNormalizer',
    preprocess_fns=['maze2d_set_terminals'],
    use_padding=False,
    max_path_length=40000,
    renderer = 'utils.rendering.Maze2dRenderer'
)

In [4]:
from diffuser.utils.config import Config

In [6]:
# Taking them from the config file (config.maze2d)
dataset_config = Config(
    args.loader,
    savepath=(args.savepath, 'dataset_config.pkl'),
    env=args.dataset,
    horizon=args.horizon,
    normalizer=args.normalizer,
    preprocess_fns=args.preprocess_fns,
    use_padding=args.use_padding,
    max_path_length=args.max_path_length,
)

[ utils/config ] Imported diffuser.datasets.sequence:GoalDataset

[utils/config ] Config: <class 'diffuser.datasets.sequence.GoalDataset'>
    env: maze2d-large-v1
    horizon: 256
    max_path_length: 40000
    normalizer: LimitsNormalizer
    preprocess_fns: ['maze2d_set_terminals']
    use_padding: False

[ utils/config ] Saved config to: saved/dataset_config.pkl



In [7]:
dataset = dataset_config()

NameError: name 'gym' is not defined

In [8]:
import gymnasium as gym


ModuleNotFoundError: No module named 'gymnasium'

In [2]:
import os

# Get the list of all environment variables
env_vars = os.environ

# Print the environment variables
for key, value in env_vars.items():
    print(f'{key}: {value}')

ALLUSERSPROFILE: C:\ProgramData
APPDATA: C:\Users\Beste Aydemir\AppData\Roaming
ASL.LOG: Destination=file
CHOCOLATEYINSTALL: C:\ProgramData\chocolatey
CHOCOLATEYLASTPATHUPDATE: 132662599615674197
CHROME_CRASHPAD_PIPE_NAME: \\.\pipe\crashpad_12876_CGPZDKGLJXFDESMM
COMMONPROGRAMFILES: C:\Program Files\Common Files
COMMONPROGRAMFILES(X86): C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432: C:\Program Files\Common Files
COMPUTERNAME: DESKTOP-P927SM4
COMSPEC: C:\WINDOWS\system32\cmd.exe
CONDA_DEFAULT_ENV: diffuser_test_new
CONDA_EXE: C:\Users\Beste Aydemir\anaconda3\Scripts\conda.exe
CONDA_PREFIX: C:\Users\Beste Aydemir\anaconda3\envs\diffuser_test_new
CONDA_PROMPT_MODIFIER: (diffuser_test_new) 
CONDA_PYTHON_EXE: C:\Users\Beste Aydemir\anaconda3\python.exe
CONDA_ROOT: C:\Users\Beste Aydemir\anaconda3
CONDA_SHLVL: 1
CONFIGSETROOT: C:\WINDOWS\ConfigSetRoot
DRIVERDATA: C:\Windows\System32\Drivers\DriverData
EFC_10456: 1
ELECTRON_RUN_AS_NODE: 1
FPS_BROWSER_APP_PROFILE_STRING: Internet Expl

In [1]:
import mujoco

In [23]:
render_config = Config(
    args.renderer,
    savepath=(args.savepath, 'render_config.pkl'),
    env=args.dataset,
)

ModuleNotFoundError: No module named 'imageio'

In [ ]:
renderer = render_config()

In [ ]:
observation_dim = dataset.observation_dim
action_dim = dataset.action_dim


#-----------------------------------------------------------------------------#
#------------------------------ model & trainer ------------------------------#
#-----------------------------------------------------------------------------#

model_config = Config(
    args.model,
    savepath=(args.savepath, 'model_config.pkl'),
    horizon=args.horizon,
    transition_dim=observation_dim + action_dim,
    cond_dim=observation_dim,
    dim_mults=args.dim_mults,
    device=args.device,
)

In [ ]:
diffusion_config = Config(
    args.diffusion,
    savepath=(args.savepath, 'diffusion_config.pkl'),
    horizon=args.horizon,
    observation_dim=observation_dim,
    action_dim=action_dim,
    n_timesteps=args.n_diffusion_steps,
    loss_type=args.loss_type,
    clip_denoised=args.clip_denoised,
    predict_epsilon=args.predict_epsilon,
    ## loss weighting
    action_weight=args.action_weight,
    loss_weights=args.loss_weights,
    loss_discount=args.loss_discount,
    device=args.device,
)

In [ ]:
trainer_config = Config(
    utils.Trainer,
    savepath=(args.savepath, 'trainer_config.pkl'),
    train_batch_size=args.batch_size,
    train_lr=args.learning_rate,
    gradient_accumulate_every=args.gradient_accumulate_every,
    ema_decay=args.ema_decay,
    sample_freq=args.sample_freq,
    save_freq=args.save_freq,
    label_freq=int(args.n_train_steps // args.n_saves),
    save_parallel=args.save_parallel,
    results_folder=args.savepath,
    bucket=args.bucket,
    n_reference=args.n_reference,
    n_samples=args.n_samples,
)

In [ ]:
model = model_config()

diffusion = diffusion_config(model)

trainer = trainer_config(diffusion, dataset, renderer)

In [ ]:
utils.report_parameters(model)

print('Testing forward...', end=' ', flush=True)
batch = utils.batchify(dataset[0])
loss, _ = diffusion.loss(*batch)
loss.backward()
print('✓')